<h1>Product Analysis for Repeat Buyers</h1>

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
#change source file path
df = pd.read_csv('C:/Users/toshiba/Desktop/AT&R/FP_src_prod.csv')
df.dtypes

Order ID                 int64
Order Date              object
Full Name (Billing)     object
Phone (Billing)         object
Item ID                float64
Item Cost              float64
Item Name               object
dtype: object

In [3]:
df['Customer ID'] = df['Phone (Billing)'].str[-7:]                 #this picks last 7 digits as ID
df['Customer ID'] = 'FP' + df['Customer ID'].astype(str)           #this adds company prefix to the ID, 'FP' in this case

In [4]:
#Remove customers who have made less than 2 purchases
df_rpbuyers = df.groupby('Customer ID').agg({'Order ID': lambda y: len(y.unique())})
df_rpbuyers = df_rpbuyers.drop(df_rpbuyers[df_rpbuyers['Order ID'] < 2].index)

In [5]:
df.dtypes

Order ID                 int64
Order Date              object
Full Name (Billing)     object
Phone (Billing)         object
Item ID                float64
Item Cost              float64
Item Name               object
Customer ID             object
dtype: object

In [6]:
#if Order data is not of date type, then change its data type
df['Order Date'] = df['Order Date'].astype('datetime64[ns]')
df.dtypes

Order ID                        int64
Order Date             datetime64[ns]
Full Name (Billing)            object
Phone (Billing)                object
Item ID                       float64
Item Cost                     float64
Item Name                      object
Customer ID                    object
dtype: object

In [7]:
#Smallest Order Date
df1 = df.groupby('Customer ID')['Order Date'].nsmallest(1).groupby(level='Customer ID').last() 

#Second Smallest Order Date
df2 = df.groupby('Customer ID')['Order Date'].nsmallest(2).groupby(level='Customer ID').last()   

#Third Smallest Order Date
df3 = df.groupby('Customer ID')['Order Date'].nsmallest(3).groupby(level='Customer ID').last()

#Fourth Smallest Order Date
df4 = df.groupby('Customer ID')['Order Date'].nsmallest(4).groupby(level='Customer ID').last()

In [8]:
dfs = [df1, df2, df3, df4, df_rpbuyers]

In [9]:
#join all dataframes on the basis of customer id
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Customer ID'], how='inner'), dfs)

In [10]:
#rename columns
df_merged.columns = ['First Order Date', 'Second Order Date', 
                     'Third Order Date', 'Fourth Order Date', 'Number of Orders Placed']


In [11]:
for i,row in df_merged.iterrows():
    if df_merged.loc[i,"Number of Orders Placed"] == 2: 
        df_merged.loc[i,"Third Order Date"] = ' '
        df_merged.loc[i, 'Fourth Order Date'] = ' '
    elif df_merged.loc[i,"Number of Orders Placed"] == 3:  
        df_merged.loc[i, "Fourth Order Date"] = ' '

In [12]:
df_joined = df.join(df_merged, on='Customer ID', how='inner')
first_order_item = df_joined[['Customer ID', 'Item ID', 'Item Name']].loc[df_joined['Order Date'] == df_joined['First Order Date']]
second_order_item = df_joined[['Customer ID', 'Item ID', 'Item Name']].loc[df_joined['Order Date'] == df_joined['Second Order Date']]
third_order_item = df_joined[['Customer ID', 'Item ID', 'Item Name']].loc[df_joined['Order Date'] == df_joined['Third Order Date']]
fourth_order_item = df_joined[['Customer ID', 'Item ID', 'Item Name']].loc[df_joined['Order Date'] == df_joined['Fourth Order Date']]
mydfs = [first_order_item, second_order_item, third_order_item, fourth_order_item]
finaldf = reduce(lambda  left,right: pd.merge(left,right,on=['Customer ID'], how='outer'), mydfs)

In [13]:
finaldf.columns = ['Customer ID', 'ITEM 1', 'First Item Name', 'ITEM 2', 'Second Item Name', 'ITEM 3', 'Third Item Name', 'ITEM 4', 'Fourth Item Name']

In [14]:
#change destination file name
finaldf.to_csv('C:/Users/toshiba/Desktop/AT&R/FP_dest_prod.csv')